In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from sklearn.svm import SVC
import pickle
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

In [ ]:
class run_machine:
    def __init__(self, file_to_be_train):
        self.file_to_be_train = file_to_be_train
        
    def clean_data(self, file_to_be_train):
        df = pd.read_csv(self.file_to_be_train)
        

        col = [col for col in df.columns]
        df.dropna(subset=col, inplace=True)

        # replace the string inside salary into 'B' and 'M"
        df["salary"] = df["salary"].replace(['Less than 1K','1K to 2K','2K to 3K','3K to 4K','4K to 5K'],'B')
        df["salary"] = df["salary"].replace(['5K to 6K','7K to 8K','8K to 9K','9K to 10K','10K or more'],"M")

        df['kids_spending'] = df['kids_spending'].str.replace(' ', "")
        df['house_utility'] = df['house_utility'].str.replace(' ', "")

        # replace string inside 'person_living_in_house' feature
        df["person_living_in_house"] = df["person_living_in_house"].replace(['10 or more'],'10')

        return df

    def column_transformer(self):
        ohe = OneHotEncoder()

        column_trans = make_column_transformer(
        (ohe, ['race', 'gender', 'employment', 'education', 'married',
        'house_type', 'house_value','vehicle', 'transport_use']), remainder='passthrough')

        return column_trans

    def my_pipeline(self):
        column_trans = self.column_transformer()
        svm = SVC(random_state=42, C=1, gamma='scale')
        pipe = make_pipeline(column_trans, svm) 
        return pipe 
    
    def execute_SVM(self, predicted_file):
        '''
        Create  model using surveyA.csv
        '''
        self.predicted_file = predicted_file
        # for train dataset
        
        df_clean = self.clean_data(self.file_to_be_train)

        # X_value
        X = df_clean.drop('salary', axis=1)

        # y_value
        y = df_clean['salary']

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y) # add stratify to ensure the fairness of distribution

        # pipeline
        pipe = self.my_pipeline()
        # model learn the data 
        
        pipe.fit(X_train, y_train)                      # here fit_transform already implemented, transform occur in pipeline
        
        # ---------------------------------------------------------------

        # for new dataset to be predicted
        # method:
        # 1- clean the data using clean_data()
        # 2- predict using pipe.predict(data)
        df_clean_pred = self.clean_data(self.predicted_file)
        X_from_pred = df_clean_pred.drop('salary', axis=1)
        
        y_predict_new = pipe.predict(X_from_pred)

    
        return y_predict_new

    def create_pickle(self):
        # for train dataset
        df_clean = self.clean_data(self.file_to_be_train)

        # X_value
        X = df_clean.drop('salary', axis=1)

        # y_value
        y = df_clean['salary']

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
        pipe = self.my_pipeline()
        pipe.fit(X_train, y_train)
        my_pickle = pickle.dump(pipe, open("xxx_ML_Ammar_1.pkl", 'wb'))

        return my_pickle

    def execute_pickle(self, predicted_file):

        self.predicted_file = predicted_file
        df_clean_pred = self.clean_data(self.predicted_file)
        #return df_clean
        X = df_clean_pred.drop('salary', axis=1)
        loaded_model = pickle.load(open("xxx_ML_Ammar_1.pkl", 'rb'))
        result = loaded_model.predict(X)
        return result

#input_test_1 = run_machine("surveyA.csv")

#input_test_1.execute_SVM("surveyA_sample.csv")
#input_test_1.create_pickle()

exe_pickle = run_machine("surveyA.csv")
exe_pickle.execute_pickle("surveyA_sample.csv")
    

# Create pickle model

In [ ]:
# generate pickle model 
def create_pickle(file_to_be_train):
    # for train dataset
    df_clean = clean_data(file_to_be_train)

    # X_value
    X = df_clean.drop('salary', axis=1)

    # y_value
    y = df_clean['salary']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    pipe = my_pipeline()
    pipe.fit(X_train, y_train)
    my_pickle = pickle.dump(pipe, open("INVOKE_ML_Ammar.pkl", 'wb'))

    return my_pickle

create_pickle("surveyA.csv")

In [ ]:
def pred_new_data(new_data):
    df_clean = clean_data(new_data)
    X = df_clean.drop('salary', axis=1)
    loaded_model = pickle.load(open("INVOKE_ML_Ammar.pkl", 'rb'))
    
    result = loaded_model.predict(X)
    return result

pred_new_data("surveyA_sample.csv")